In [1]:
!which python

/lfs/skampere1/0/kaif/conda/envs/TheoremSense/bin/python


In [5]:
import google.generativeai as genai
import os
from tqdm import tqdm

genai.configure(api_key=os.environ["GOOGLE_AI_API_KEY"])

# Set up the model
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


def openai_to_google(messages):
    return [
        {
            "role": "user" if message['role'] == "user" else "model",
            "parts": [message['content']]
        }
        for message in messages
    ]


def google_generate(inputs):
    return [model.generate_content(openai_to_google(input)).text for input in tqdm(inputs)]

In [2]:
%%capture
from dataset import load_datasets, BOXED_ANSWERS_DATASETS
import numpy as np
import json

# load the name maps
with open('name_maps.json', 'r') as f:
    name_maps = json.load(f)
    DATASET_MAP = name_maps['DATASET_MAP']
    MODEL_MAP = name_maps['MODEL_MAP']

datasets_raw = load_datasets(BOXED_ANSWERS_DATASETS)
datasets = {
    DATASET_MAP[dataset['name']]: dataset['data']['train']
    for dataset in datasets_raw
}

np.random.seed(0)
num_samples = 100
datasets_subset = {k: np.random.choice(v, num_samples, replace=False) for k, v in datasets.items()}

## Split Proofs Into Steps

In [6]:
import re
from util import generate, filter_errors, save_output, Prompt

split_proof_chat = Prompt([
    {
        "role": "user",
        "content": "PROOF: Let $p$ be the number of pie crusts, and let $f$ be the amount of flour per crust. Because the total amount of flour needs to remain constant, we can express the relationship between pie crusts as $p\\cdot f = c$, where $c$ is a constant value.Since we know that 30 pie crusts each use up $\\frac16$ cup of flour, $30\\left(\\frac16\\right)=c$ or $c=5$. When $p=20$, the equation becomes $20\\cdot f=5$, or $f=\\frac5{20}=\\boxed{\\frac14}$",
    },
    {
        "role": "system",
        "content": "SPLIT: <step>Let $p$ be the number of pie crusts, and let $f$ be the amount of flour per crust.</step>\n<step>Because the total amount of flour needs to remain constant, we can express the relationship between pie crusts as $p\cdot f = c$, where $c$ is a constant value.</step>\n<step>Since we know that 30 pie crusts each use up $\\frac16$ cup of flour, $30\left(\\frac16\\right)=c$ or $c=5$.</step>\n<step>When $p=20$, the equation becomes $20\cdot f=5$.</step>\n<step>So solving for $f$, we get $f=\\frac5{20}=\\boxed{\\frac14}$.</step>",
    },
    {
        "role": "user",
        "content": "PROOF: We begin by multiplying out the denominator and then  squaring both sides \\begin{align*}\\frac{\\sqrt{5x}}{\\sqrt{3(x-1)}}&=2\\\\(\\sqrt{5x})^2 &=\\left(2\\sqrt{3(x-1)}\\right)^2\\\\5x &= 12(x-1)\\\\12& =7x\\\\x&=\\boxed{\\frac{12}{7}}.\\\\\\end{align*}Checking, we see that this value of $x$ satisfies the original equation, so it is not an extraneous solution.",
    },
    {
        "role": "system",
        "content": "SPLIT: <step>We begin by multiplying out the denominator and then squaring both sides, resulting in $\\frac{\sqrt{5x}}{\sqrt{3(x-1)}}=2$.</step>\n<step>The equation simplifies to $(\sqrt{5x})^2 =\left(2\sqrt{3(x-1)}\\right)^2$.</step>\n<step>Which further simplifies to $5x = 12(x-1)$.</step>\n<step>From which we get $12 = 7x$.</step>\n<step>Solving for $x$ gives us $x=\\boxed{\\frac{12}{7}}$.</step>\n<step>Checking, we see that this value of $x$ satisfies the original equation, so it is not an extraneous solution.</step>",
    },
    {
        "role": "user",
        "content": "PROOF: There is a point of intersection for each $x$ such that $f(x^2)=f(x^4)$. Since $f$ is invertible, this equation is satisfied only if $x^2=x^4$, so we simply count solutions to that equation. We can rearrange the equation $x^2=x^4$ as follows: \\begin{align*}0 &= x^4-x^2 \\\\0 &= x^2(x^2-1) \\\\0 &= x^2(x+1)(x-1)\\end{align*}The last factorization shows that the solutions are $x=-1,0,1$. Therefore, the graphs of $y=f(x^2)$ and $y=f(x^4)$ must intersect at exactly $\\boxed{3}$ points.",
    },
    {
        "role": "system",
        "content": "SPLIT: <step>There is a point of intersection for each $x$ such that $f(x^2)=f(x^4)$.</step>\n<step>Since $f$ is invertible, this equation is satisfied only if $x^2=x^4$, so we simply count solutions to that equation.</step>\n<step>We can rearrange the equation $x^2=x^4$ as follows: $0 = x^4-x^2$.</step>\n<step>Factoring out $x^2$, we get $0 = x^2(x^2-1)$.</step>\n<step>Which leads to $0 = x^2(x+1)(x-1)$, showing that the solutions are $x=-1,0,1$.</step>\n<step>Therefore, the graphs of $y=f(x^2)$ and $y=f(x^4)$ must intersect at exactly $\\boxed{3}$ points.</step>",
    },
    {
        "role": "user",
        "content": "PROOF: Let $c$ equal the cost of a bouquet of 39 roses (in dollars). Since we know that the price of a bouquet is directly proportional to the number of roses it contains, we can set up the following proportion\n \\begin{align*}\n\\frac{c}{39}&=\\frac{20}{12}\\\\\n\\Rightarrow \\qquad c&=\\left(\\frac{20}{12}\\right)(39)\\\\\n\\Rightarrow \\qquad c&=\\boxed{65}\n\\end{align*}"
    },
    {
        "role": "system",
        "content": "SPLIT: <step>Let $c$ equal the cost of a bouquet of 39 roses (in dollars).</step>\n<step>Since we know that the price of a bouquet is directly proportional to the number of roses it contains, we can set up the proportion $\\frac{c}{39}&=\\frac{20}{12}$.</step>\n<step>Solving for $c$ yields $c=\\left(\\frac{20}{12}\\right)(39)$ or $c=\\boxed{65}$</step>",
    },
])


@filter_errors
def split_proof(proof):
    steps = re.findall(r'<step>(.*?)</step>', proof, re.DOTALL)
    # validate that the proof was split correctly
    # assumes number of steps is at least 2
    if len(steps) < 2:
        raise ValueError(f"Not enough steps in proof found.\n{proof}")
    # make sure there is nothing before or after the steps
    if not proof.endswith("</step>"):
        raise ValueError(f"Proof does not end with a step.\n{proof}")
    return steps


questions = []
for name, dataset in datasets_subset.items():
    for question in dataset:
        proof = question['answer']
        questions.append(split_proof_chat + [{"role": "user", "content": proof}])

splits = generate(questions, google_generate, split_proof, max_attempts=30)